# Text Preprocessing

## 1️⃣ **Lowercasing**

### 🔍 **Why:**
- Words like "Good", "GOOD", and "good" are semantically identical but are treated as separate tokens if not lowercased.


### 📌 **Real-World:**
- In a product review system, "Excellent" and "excellent" should be considered the same sentiment word.

In [ ]:
text = "Good Morning EVERYONE! Let's Start Our NLP Journey."
lowercased = text.lower()

print("🔵 Original:", text, "\n")
print("🟢 Preprocessed:", lowercased)

🔵 Original: Good Morning EVERYONE! Let's Start Our NLP Journey. 

🟢 Preprocessed: good morning everyone! let's start our nlp journey.


---

## 2️⃣ **Remove HTML Tags**

### 🔍 **Why:**
- HTML is markup used for layout, not meaning.

- If you're scraping websites (news, blogs), tags like div tag, span tag add noise.

### 📌 **Real-World:**
- When extracting articles from news sites, you'll find lots of formatting tags. Models get confused by p tag, a href tag etc.

In [2]:
from bs4 import BeautifulSoup
import regex as re

# sample HTML text
text = "<div>Hello <b>World</b>! NLP is <i>awesome</i>.</div>"

# Using BeautifulSoup to remove HTML tags
cleaned = BeautifulSoup(text, "html.parser").get_text()

# Using regex to remove HTML tags
re_pattern = "<(?:\"[^\"]*\"['\"]*|'[^']*'['\"]*|[^'\">])+>" # # Remove all HTML tags from a string
cleaned_re = re.sub(re_pattern, '', text)
# cleaned = cleaned.strip()

print("🔵 Original:", text, "\n")
print("🟢 Preprocessed with BS4:", cleaned, "\n")
print("🟢 Preprocessed with regex:", cleaned_re)

🔵 Original: <div>Hello <b>World</b>! NLP is <i>awesome</i>.</div> 

🟢 Preprocessed with BS4: Hello World! NLP is awesome. 

🟢 Preprocessed with regex: Hello World! NLP is awesome.


---

## 3️⃣ **Remove URLs**

### 🔍 **Why:**
- URLs usually don’t carry semantic meaning.

- They are high variance strings (each one is unique) and hurt model generalization.

### 📌 **Real-World:**
- In a tweet like “Check this out 👉 https://xyz.com”, we care more about the sentiment or emotion, not the URL itself.

In [ ]:
# it will remove any type of URL from the text just apply on your text
import regex as re

# Example usage:
text = "Visit our website at https://www.example.com/page.html or check out www.another-site.org. You can also find info at mydomain.net/info."

# Regex pattern to match various URL formats (http, https, www, without scheme)
url_pattern = re.compile(
    r'https?://[^\s/$.?#].[^\s]*'  # Matches http/https URLs
    r'|www\.[^\s/$.?#].[^\s]*'    # Matches www. URLs
    r'|[a-zA-Z0-9.-]+\.[a-zA-Z]{2,6}(?:/[^\s]*)?' # Matches domain-only URLs like example.com
)

# Remove URLs from the text
text_without_urls= url_pattern.sub(r'', text)

print("🔵 Original:", text, "\n")
print("🟢 Preprocessed:", text_without_urls)

🔵 Original: Visit our website at https://www.example.com/page.html or check out www.another-site.org. You can also find info at mydomain.net/info. 

🟢 Preprocessed: Visit our website at  or check out  You can also find info at 


---

## 4️⃣ **Remove Punctuation**

### 🔍 **Why:**
-  Punctuation marks like `!`, `.`, `?` create unnecessary tokens.

- Removing them helps reduce noise, especially in tasks like text classification or topic modeling.

| Method                               | Removes ASCII | Removes Unicode | Customizable |
| ------------------------------------ | ------------- | --------------- | ------------ |
| `string.punctuation` + `translate()` | ✅ Yes         | ❌ No            | ❌ Limited    |
| `re.sub(r'\p{P}+', '', text)`       | ✅ Yes         | ✅ Yes           | ✅ Yes        |

- string.punctuation only covers ASCII punctuation & returns: <div style=color:yellow;>!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~</div>

- Unicode symbols like  `“`, `—`, and `…` are not included in string.punctuation, so we need to add them manually or using regex


### 📌 **Real-World:**
- For spam detection or document classification, punctuation doesn’t usually help (unless you’re analyzing writing style).

In [12]:
import string

text = "Hello!!! How are you??? I'm fine:) “Great”— yes…"
no_punc = text.translate(str.maketrans('', '', string.punctuation + '“”—…'))

print("🔵 Original:", text, "\n")
print("🟢 Preprocessed:", no_punc)

🔵 Original: Hello!!! How are you??? I'm fine:) “Great”— yes… 

🟢 Preprocessed: Hello How are you Im fine Great yes


In [13]:
import regex as re

text = "Hello!!! How are you??? I'm fine:) “Great”— yes…"
no_punc = re.sub(r'\p{P}+', '', text)

print("🔵 Original:", text, "\n")
print("🟢 Preprocessed:", no_punc)

🔵 Original: Hello!!! How are you??? I'm fine:) “Great”— yes… 

🟢 Preprocessed: Hello How are you Im fine Great yes


---

## 5️⃣ **Chat Word Treatment (e.g., GN → Good Night)**

### 🔍 **Why:**
- Chat language is full of abbreviations: "lol", "idk", "smh".

- They need to be normalized to standard English for models to understand.

### 📌 **Real-World:**
- In customer support or social media, replacing abbreviations like `"brb"` with `"be right back"` helps understand the message better.

### 🗣️ **Slang Words**
- The following two GitHub repositories provide comprehensive lists of slang words and their standard English equivalents.These resources are useful for expanding chat abbreviations (e.g., "GN" → "Good Night") during text preprocessing

    - [Repo1](https://github.com/ipekdk/abbreviation-list-english)
    - [Repo2](https://github.com/bodhwani/NLP-VIT-BOT/blob/master/slangs.csv)
    - [Comman Slang Words](https://github.com/rishabhverma17/sms_slang_translator/blob/master/slang.txt)

#### Replace Slang words with the help of tokenization

In [87]:
# Use spaCy for tokenization and keep punctuation marks
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_sm")
text = "BRB guys, lol this is funny. ttyl!"
# Ensure 'nlp' is already loaded with spacy.load("en_core_web_sm")
doc = nlp(text)  # 'text' variable is already defined

# Load slang dictionary from CSV
chat_dict = pd.read_csv("slangs.csv")  # Must have columns 'Abbr', 'Fullform'
slang_map = dict(zip(chat_dict['Abbr'].str.lower(), chat_dict['Fullform']))

# Replace slangs with full forms, preserving punctuation
processed_tokens = []
for token in doc:
    token_text = token.text
    lower_token = token_text.lower()
    if lower_token in slang_map:
        processed_tokens.extend(slang_map[lower_token].split())
    else:
        processed_tokens.append(token_text)

preprocessed_text = " ".join(processed_tokens)

print("🔵 Original Text:")
print(text)
print("\n🟢 Preprocessed Text:")
print(preprocessed_text)


🔵 Original Text:
BRB guys, lol this is funny. ttyl!

🟢 Preprocessed Text:
Be Right Back guys , Laughing Out Loud this is funny . Talk To You Later !


#### Replace Slang word with the help of Split

- you can see the difference `','` , `'.'` and `'!'` present in original text but not in preprocessed text using split but in the case of tokenization there is no missingness of `','` , `'.'` and `'!'`

In [88]:
# just apply to your text this comprehenisve list of slang abbreviations
# Make sure to have a CSV file named 'slangs.csv' with columns 'Abbr

import pandas as pd
import regex as re  # Use the 'regex' module for Unicode property escapes

# Load slang dictionary from CSV
chat_dict = pd.read_csv("slangs.csv")  # Must have columns 'Abbr', 'Fullform'
slang_map = dict(zip(chat_dict['Abbr'].str.lower(), chat_dict['Fullform']))

# Sample chat-like text
text = "BRB guys, lol this is funny. ttyl!"

# Remove punctuation using regex for matching text
text_clean = re.sub(r'\p{P}+', '', text)

# Tokenize by whitespace
words = text_clean.split()

# Replace slangs
chat_fixed = []
for word in words:
    word_lower = word.lower()
    if word_lower in slang_map:
        chat_fixed.append(slang_map[word_lower])
    else:
        chat_fixed.append(word)

# Join the cleaned text
cleaned_text = " ".join(chat_fixed) # The join() method takes all items in an iterable and joins them into one string. A string must be specified as the separator

# Output
print("🔵 Original:", text, "\n")
print("🟢 Preprocessed:", cleaned_text)

🔵 Original: BRB guys, lol this is funny. ttyl! 

🟢 Preprocessed: Be Right Back guys Laughing Out Loud this is funny Talk To You Later


---

## 6️⃣ **Spelling Correction**

### 🔍 **Why:**
- Typos are common in user input (especially social media).

- Words like “beleive” won’t match “believe” in dictionaries or embeddings.

### 📌 **Real-World:**
- Spell correction boosts chatbot understanding and auto-correction in search bars.

In [7]:
from autocorrect import Speller
from textblob import TextBlob
from spellchecker import SpellChecker


spell = Speller(lang='en')
text = "I realy love natral language prosesing."

# Autocorrect
corrected_autocorrect = spell(text)

# TextBlob
corrected_textblob = str(TextBlob(text).correct())

# SpellChecker
spellchecker = SpellChecker()
# Tokenize the text for word-level correction
tokens = text.split()
corrected_pyspellchecker = ' '.join([spellchecker.correction(word) or word for word in tokens])


print("🔵 Original:", text, "\n")
print("🟢 Preprocessed (autocorrect):", corrected_autocorrect, "\n")
print("🟢 Preprocessed (TextBlob):", corrected_textblob, "\n")
print("🟢 Preprocessed (pyspellchecker):", corrected_pyspellchecker)

🔵 Original: I realy love natral language prosesing. 

🟢 Preprocessed (autocorrect): I really love natural language crossing. 

🟢 Preprocessed (TextBlob): I really love natural language pressing. 

🟢 Preprocessed (pyspellchecker): I really love natural language prosesing.


---

## 7️⃣ **Remove Stop Words**

### 🔍 **Why:**
- Stopwords like "the", "is", "at" are function words, not content words.

- Removing them improves focus on important tokens.

### 📌 **Real-World:**
- In a movie review, “The movie was absolutely fantastic”, word “fantastic” carries the sentiment, not “the”.

In [7]:
# Remove stop words from tokens using spaCy
import spacy
nlp = spacy.load("en_core_web_sm")
text = "This is a sample sentence with some stop words"
doc = nlp(text)
tokens = [token.text for token in doc]
stop_words = nlp.Defaults.stop_words
filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
print(filtered_tokens)


['sample', 'sentence', 'stop', 'words']


In [11]:
print(len(stop_words))

326


In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')

text = "This is a sample sentence."
tokens = word_tokenize(text)
filtered = [word for word in tokens if word.lower() not in stopwords.words('english')]


In [ ]:
# sample big text
text = "This is a sample sentence with some stop words. It contains multiple sentences and should be filtered accordingly."


In [ ]:
import nltk
from nltk.corpus import stopwords
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Input text
text = "This is a sample sentence with some stop words. It contains multiple sentences and should be filtered accordingly."

# Tokenize using spaCy
tokens = nlp(text)

# Filter stopwords using NLTK's list
filtered = " ".join([word.text for word in tokens if word.text.lower() not in stopwords.words('english')])

print("Filtered Tokens:", filtered)



Filtered Tokens: sample sentence stop words . contains multiple sentences filtered accordingly .


In [25]:
import nltk
from nltk.corpus import stopwords
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Input text
text = "This is a sample sentence with some stop words. It contains multiple sentences and should be filtered accordingly."

# Tokenize using spaCy
tokens = nlp(text)

# Filter stopwords using NLTK's list
filtered = " ".join([token.text for token in tokens if not token.is_stop])

print("Filtered Tokens:", filtered)


Filtered Tokens: sample sentence stop words . contains multiple sentences filtered accordingly .


In [17]:
# Import library
import spacy

# Load the language model
nlp = spacy.load("en_core_web_sm")

# Process the text
text = "This is a sample sentence with some stop words"
doc = nlp(text)

# Remove stop words
filtered_tokens = " ".join([token.text for token in doc if not token.is_stop])

# Print the text excluding stop words
print(filtered_tokens)

sample sentence stop words


| Feature       | NLTK                             | spaCy                                   |
| ------------- | -------------------------------- | --------------------------------------- |
| Stopword List | Manually maintained              | Linguistically curated                  |
| Tokenization  | Basic (word\_tokenize)           | Advanced (handles context, punctuation) |
| Flexibility   | High (can customize list easily) | High (custom rules, pipelines)          |
| Performance   | Lightweight                      | Heavier but more powerful               |


---

## 8️⃣ **Handling Emojis**

### 🔍 **Why:**
- Emojis are sentiment-rich tokens (😊, 😢).

- You can either remove them, convert to words, or use them as features.

### 📌 **Real-World:**
- In social media sentiment analysis, 😠 and ❤️ change the tone completely and must be considered.

In [72]:
import emoji
import regex as re

text = "Good job!👍I’m so happy😊"
emoji_text = emoji.demojize(text)

# Remove ':' and '_' characters
emoji_text_cleaned = emoji_text.replace(':', ' ').replace('_', ' ')

print(f"🔵 Original: {text}\n")
print(f"🟢 Preprocessed: {emoji_text_cleaned}")


🔵 Original: Good job!👍I’m so happy😊

🟢 Preprocessed: Good job! thumbs up I’m so happy smiling face with smiling eyes 


---

## 9️⃣ **Tokenization**

### ⚠️ **Very Important Step**

### 🔍 **Why:**
Machine learning models in NLP typically require numerical input. Tokenization converts text into a format that can be converted into numerical representations (like word embeddings) that models can understand.

#### **Example:**
Let's say we have the sentence: `The quick brown fox jumps over the lazy dog.`
Word Tokenization:
> ["The", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog", "."]

### 📌 **Real-World:**
- In the search engine, "best Italian restaurants near me" is tokenized into the words: ["best", "Italian", "restaurants", "near", "me"].

In [ ]:
from nltk.tokenize import word_tokenize

text = "Tokenization splits sentences into words."
tokens = word_tokenize(text)

print("🔵 Original:", text)
print("🟢 Preprocessed:", tokens)


---

## 🔟 **Stemming**

### 🔍 **Why:**
- Stemming reduces words to a base/root form. It’s fast and works well in information retrieval systems.

- May produce non-words (e.g., “studies” → “studi”).

### 📌 **Real-World:**
- Used in search engines (e.g., “searching”, “searched”, “searches” → “search”) to show relevant documents.

In [ ]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
text = "We are studying various stemmed words: running, studies, cries"
tokens = word_tokenize(text)
stemmed = [stemmer.stem(w) for w in tokens]

print("🔵 Original:", tokens)
print("🟢 Preprocessed:", stemmed)


---

## 🔢 **Lemmatization**

### 🔍 **Why:**
- Like stemming, but linguistically accurate.

- Uses vocabulary and grammar rules to return the correct base word.

### 📌 **Real-World:**
- For grammar-sensitive tasks like question answering or summarization, “was” should be lemmatized to “be”, not “wa”.

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

text = "The children were playing with their better toys."
doc = nlp(text)
lemmatized = [token.lemma_ for token in doc]

print("🔵 Original:", text)
print("🟢 Preprocessed:", lemmatized)


## ✅ Summary Table

| Step                | Purpose                                                                 | Use Case Examples                                     |
|---------------------|-------------------------------------------------------------------------|--------------------------------------------------------|
| Lowercasing          | Normalize text casing                                                   | Text classification, search                           |
| Remove HTML Tags     | remove unwanted HTML tags                                                | Web scraping, email cleaning                          |
| Remove URLs          | Remove unnecessary web links                                             | Social media, forums                                  |
| Remove Punctuation   | Clean punctuation noise                                                  | Preprocessing for BoW/TF-IDF                          |
| Chat Word Treatment  | Convert slang to standard English like GD to Good Night                  | Chatbots, social media analysis                       |
| Spelling Correction  | Fix typos for vocabulary consistency                                     | User reviews, text input                              |
| Remove Stop Words    | Focus on meaningful words                                                 | Summarization, topic modeling                         |
| Handle Emojis        | Preserve or convert emojis based on task                                | Sentiment analysis                                    |
| Tokenization         | Break down text into tokens                                              | All NLP tasks                                         |
| Stemming             | Reduce word forms to base/root                                           | Search engines, topic modeling                        |
| Lemmatization        | Get accurate root word using grammar                                     | Parsing, QA systems, deep learning models             |
